In [1]:
import pandas as pd

In [27]:
df = pd.DataFrame({'a':[100,200,100,200],'b':[100,200,300,400]})

In [28]:
df.head()

,a,b
0,100,100
1,200,200
2,100,300
3,200,400


In [29]:
df_calc = df.copy()

In [30]:
df_calc['percentage_sharp_drop'] = (((df.iloc[-2]['a'] - df.iloc[-1]['a']) / df.iloc[-2]['a'] ) * 100)

In [31]:
df_calc.head()

,a,b,percentage_sharp_drop
0,100,100,-100.0
1,200,200,-100.0
2,100,300,-100.0
3,200,400,-100.0


In [23]:
abs(df.iloc[-2]['a'] - df.iloc[-1]['a'] ) 

100

In [24]:
 df.iloc[-1]['a']

400

In [18]:
100/400

0.25

In [ ]:
1530